# Poker Bankroll Decision System

Turn messy live poker logs into win rate and variance estimates, run risk of ruin and drawdown simulations, and output a clear stake recommendation for the current bankroll and conditions.

## Notebook Overview

This notebook implements a complete bankroll decision system with four main sections:

1. **Import** - Load and normalize session data from various sources
2. **Enrich** - Derive features and enhance data with meaningful poker metrics
3. **Estimate** - Calculate win rates, variance, and confidence intervals by stake and conditions
4. **Simulate + Report** - Run Monte Carlo simulations and generate stake recommendations

---

## Section 1: Import

Import necessary libraries and load poker session data from various sources. This section handles data ingestion and initial normalization into our canonical schema.

In [ ]:
# Import libraries and our poker bankroll analysis modules
import sys
from pathlib import Path

# Add src directory to path
PROJECT_ROOT = Path('/Users/johnsteill/PROJ/Poker_RoR')
sys.path.append(str(PROJECT_ROOT / 'src'))

# Import our poker analysis modules
from poker_bankroll import PokerBankrollAnalyzer, quick_analysis

# Basic data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("✅ Libraries and modules imported successfully")
print(f"📁 Project root: {PROJECT_ROOT}")

# Initialize the analyzer
analyzer = PokerBankrollAnalyzer(PROJECT_ROOT)

✅ Libraries imported successfully


In [ ]:
# Display current configuration
print("⚙️ Current Configuration:")
print("=" * 30)

config = analyzer.config
print(f"🎲 Simulations: {config['simulation']['n_simulations']:,}")
print(f"💰 Current Bankroll: {config['simulation']['current_bankroll_bb']:,} BB")
print(f"⚠️ Risk Tolerance: {config['simulation']['risk_tolerance']:.1%}")
print(f"⏱️ Time Horizons: {config['simulation']['time_horizons']}")

# You can modify the configuration here if needed
# analyzer.config['simulation']['current_bankroll_bb'] = 3000  # Example: change bankroll
# analyzer.config['simulation']['risk_tolerance'] = 0.10      # Example: change risk tolerance

print("\n? Tip: You can modify analyzer.config to adjust parameters before running analysis")

In [ ]:
# Import session data
raw_sessions = analyzer.import_data()

print(f"\n📊 Session Data Overview:")
print(f"Total sessions: {len(raw_sessions)}")
print(f"Stakes played: {raw_sessions['stake_text'].value_counts().to_dict()}")
print(f"Total hours: {raw_sessions['hours_played'].sum():.1f}")

# Show sample of the data
print(f"\n📋 Sample sessions:")
display_cols = ['date', 'stake_text', 'hours_played', 'buyins_usd', 'cashouts_usd', 'room']
raw_sessions[display_cols].head()

In [ ]:
# Quick data quality check
raw_sessions['net_result'] = raw_sessions['cashouts_usd'] - raw_sessions['buyins_usd']

print("📈 Basic Statistics:")
print(f"Win rate per session: ${raw_sessions['net_result'].mean():.2f} ± ${raw_sessions['net_result'].std():.2f}")
print(f"Total net result: ${raw_sessions['net_result'].sum():,.2f}")
print(f"Winning sessions: {(raw_sessions['net_result'] > 0).sum()}/{len(raw_sessions)} ({(raw_sessions['net_result'] > 0).mean():.1%})")

# Simple visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Session results by stake
raw_sessions.boxplot(column='net_result', by='stake_text', ax=ax1)
ax1.set_title('Session Results by Stake')
ax1.set_xlabel('Stake')
ax1.set_ylabel('Net Result ($)')

# Hours played distribution
raw_sessions['hours_played'].hist(bins=20, ax=ax2)
ax2.set_title('Hours Played Distribution')
ax2.set_xlabel('Hours')
ax2.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

---

## Section 2: Enrich

Derive features that matter for variance analysis: effective big blinds, straddle impact, side game exposure, and stack depth effects. This section enhances the raw data with meaningful poker metrics.

In [ ]:
# Enrich session data with derived features
enriched_sessions = analyzer.enrich_data()

print("🔧 Data Enrichment Summary:")
print(f"Original columns: {len(raw_sessions.columns)}")
print(f"Enriched columns: {len(enriched_sessions.columns)}")

# Show new features
new_features = set(enriched_sessions.columns) - set(raw_sessions.columns)
print(f"New features: {sorted(new_features)}")

# Display enriched metrics
key_metrics = ['bb_per_hand', 'bb_per_hour', 'hands_played', 'side_game_intensity']
print(f"\n📊 Key Metrics Summary:")
enriched_sessions[key_metrics].describe().round(3)

In [ ]:
# Visualize enriched data
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# BB per hour by stake
enriched_sessions.boxplot(column='bb_per_hour', by='stake_text', ax=axes[0,0])
axes[0,0].set_title('BB/Hour by Stake Level')
axes[0,0].tick_params(axis='x', rotation=45)

# Side game intensity distribution
enriched_sessions['side_game_intensity'].hist(bins=20, ax=axes[0,1])
axes[0,1].set_title('Side Game Intensity Distribution')
axes[0,1].set_xlabel('Intensity Score')

# Hands per hour by conditions
enriched_sessions.boxplot(column='hands_per_hour', by='stack_depth_class', ax=axes[1,0])
axes[1,0].set_title('Hands/Hour by Stack Depth')

# Straddle impact
straddle_impact = enriched_sessions.groupby('straddle_exposure')['bb_per_hour'].mean()
straddle_impact.plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('BB/Hour by Straddle Exposure')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\n? Total hands played: {enriched_sessions['hands_played'].sum():,}")
print(f"Average BB/hour: {enriched_sessions['bb_per_hour'].mean():.2f}")
print(f"Average hands/hour: {enriched_sessions['hands_per_hour'].mean():.1f}")

---

## Section 3: Estimate

Estimate per hand win rate (μ) and variance (σ²) by stake and conditions. This section provides the statistical foundation for bankroll decisions with confidence intervals and sample sizes.

In [ ]:
# Estimate win rates and variance by stake
stake_estimates = analyzer.estimate_parameters()

print("📊 Parameter Estimates by Stake:")
print("=" * 40)

# Display key estimates
display_cols = ['stake_text', 'n_sessions', 'total_hands', 'mu_bb_per_hand', 
                'mu_bb_ci_lower', 'mu_bb_ci_upper', 'sigma2_bb_per_hand']
stake_estimates[display_cols].round(4)

In [ ]:
# Visualize win rate estimates with confidence intervals
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Win rate by stake with confidence intervals
stakes = stake_estimates['stake_text']
mu_values = stake_estimates['mu_bb_per_hand']
ci_lower = stake_estimates['mu_bb_ci_lower']
ci_upper = stake_estimates['mu_bb_ci_upper']
sample_sizes = stake_estimates['n_sessions']

ax1.errorbar(stakes, mu_values, 
             yerr=[mu_values - ci_lower, ci_upper - mu_values],
             fmt='o', capsize=5, capthick=2, markersize=8)

# Add sample sizes as text
for i, (stake, mu, n) in enumerate(zip(stakes, mu_values, sample_sizes)):
    ax1.text(i, mu + 0.001, f'n={n}', ha='center', va='bottom', fontsize=9)

ax1.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Break-even')
ax1.set_xlabel('Stake Level')
ax1.set_ylabel('Win Rate (BB/hand)')
ax1.set_title('Win Rate by Stake Level\\n(with 95% Confidence Intervals)')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot 2: Variance by stake
variance_values = stake_estimates['sigma2_bb_per_hand']
bars = ax2.bar(stakes, variance_values, alpha=0.7, color='orange')

# Add variance values on bars
for bar, var in zip(bars, variance_values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{var:.4f}', ha='center', va='bottom', fontsize=9)

ax2.set_xlabel('Stake Level')
ax2.set_ylabel('Variance (BB²/hand)')
ax2.set_title('Variance by Stake Level')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate Sharpe-like ratios
stake_estimates['sharpe_ratio'] = stake_estimates['mu_bb_per_hand'] / np.sqrt(stake_estimates['sigma2_bb_per_hand'])
print(f"\n📈 Risk-Adjusted Performance (Sharpe Ratios):")
for _, row in stake_estimates.iterrows():
    print(f"{row['stake_text']}: {row['sharpe_ratio']:.3f}")

In [ ]:
# Summary table for decision making
print("📋 Summary Table for Bankroll Decisions")
print("=" * 50)

summary_table = stake_estimates[['stake_text', 'n_sessions', 'total_hands', 
                                'mu_bb_per_hand', 'sigma2_bb_per_hand', 'sharpe_ratio']].copy()

# Add derived metrics
summary_table['bb_per_100_hands'] = summary_table['mu_bb_per_hand'] * 100
summary_table['std_per_hand'] = np.sqrt(summary_table['sigma2_bb_per_hand'])

# Display formatted table
summary_formatted = summary_table.round(4)
print(summary_formatted.to_string(index=False))

# Highlight key insights
best_winrate = summary_table.loc[summary_table['mu_bb_per_hand'].idxmax()]
best_sharpe = summary_table.loc[summary_table['sharpe_ratio'].idxmax()]

print(f"\n🎯 Key Insights:")
print(f"Highest win rate: {best_winrate['stake_text']} at {best_winrate['mu_bb_per_hand']:.4f} BB/hand")
print(f"Best risk-adjusted: {best_sharpe['stake_text']} with Sharpe ratio {best_sharpe['sharpe_ratio']:.3f}")

In [ ]:
# Run Monte Carlo simulations
print("🎲 Running Monte Carlo Simulations...")
print("=" * 40)

simulation_results = analyzer.run_simulations()

print(f"✅ Simulations complete for {len(simulation_results)} stakes")

# Display key simulation metrics
print(f"\n📊 Risk of Ruin (10,000 hands):")
for _, row in simulation_results.iterrows():
    ror = row.get('ror_10000h', 0)
    final_mean = row.get('final_mean_10000h', 0)
    print(f"{row['stake_text']}: {ror:.1%} RoR, Expected: {final_mean:.0f} BB")

simulation_results[['stake_text', 'mu', 'sigma', 'ror_5000h', 'ror_10000h']].round(4)

---

## Section 4: Simulate + Report

Run Monte Carlo simulations to compute risk of ruin and drawdown probabilities over various time horizons. Generate stake recommendations and decision memos based on current bankroll and risk tolerance.

In [ ]:
# Visualize simulation results
fig = go.Figure()

time_horizons = analyzer.config['simulation']['time_horizons']
risk_tolerance = analyzer.config['simulation']['risk_tolerance']

# Plot risk of ruin by time horizon for each stake
for _, row in simulation_results.iterrows():
    stake = row['stake_text']
    ror_values = [row[f'ror_{h}h'] for h in time_horizons]
    
    fig.add_trace(go.Scatter(
        x=time_horizons,
        y=ror_values,
        mode='lines+markers',
        name=f'{stake}',
        line=dict(width=2)
    ))

# Add risk tolerance line
fig.add_hline(
    y=risk_tolerance,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Risk Tolerance ({risk_tolerance:.1%})"
)

fig.update_layout(
    title="Risk of Ruin by Time Horizon",
    xaxis_title="Hands Played",
    yaxis_title="Risk of Ruin",
    yaxis_tickformat=".1%",
    height=500
)

fig.show()

# Show drawdown analysis for 10k hands
print(f"\\n⚠️ Drawdown Probabilities (10,000 hands):")
drawdown_cols = [col for col in simulation_results.columns if 'dd_' in col and '10000h' in col]
if drawdown_cols:
    dd_summary = simulation_results[['stake_text'] + drawdown_cols]
    for col in drawdown_cols:
        dd_summary[col] = dd_summary[col].apply(lambda x: f"{x:.1%}")
    print(dd_summary.to_string(index=False))

In [ ]:
# Generate final recommendations and decision memo
recommendations, decision_memo = analyzer.generate_recommendations()

print("🎯 STAKE RECOMMENDATIONS")
print("=" * 50)

# Display clean recommendations table
summary_cols = ['stake_text', 'recommendation', 'ror_10k_hands', 'expected_return_bb_per_hand']
rec_display = recommendations[summary_cols].copy()
rec_display['ror_10k_hands'] = rec_display['ror_10k_hands'].apply(lambda x: f"{x:.1%}")
rec_display['expected_return_bb_per_hand'] = rec_display['expected_return_bb_per_hand'].round(4)

print(rec_display.to_string(index=False))

# Highlight the top recommendation
recommended = recommendations[recommendations['recommendation'] == 'RECOMMENDED']
if len(recommended) > 0:
    top_rec = recommended.iloc[0]
    print(f"\\n✅ TOP RECOMMENDATION: {top_rec['stake_text']}")
    print(f"   Risk of Ruin: {top_rec['ror_10k_hands']:.1%}")
    print(f"   Expected Return: {top_rec['expected_return_bb_per_hand']:.4f} BB/hand")
    print(f"   Reason: {top_rec['reason']}")
else:
    print(f"\\n⚠️ No stakes currently meet the recommended criteria")
    print(f"   Risk tolerance: {analyzer.config['simulation']['risk_tolerance']:.1%}")
    print(f"   Current bankroll: {analyzer.config['simulation']['current_bankroll_bb']:,} BB")

In [ ]:
# Display and save decision memo
print("📝 DECISION MEMO")
print("=" * 60)
print(decision_memo)

# Save all results
results = {
    'enriched_sessions': enriched_sessions,
    'stake_estimates': stake_estimates,
    'simulation_results': simulation_results,
    'recommendations': recommendations,
    'decision_memo': decision_memo
}

analyzer.save_results(results)

print(f"\\n💾 Analysis complete! All results saved to the project directory.")

---

## Summary & Next Steps

This notebook provides a complete bankroll decision system for live poker players. Here's what we've accomplished:

### ✅ What This Notebook Does

1. **Import & Normalize**: Loads poker session data and converts it to a canonical schema
2. **Enrich & Analyze**: Derives meaningful features like effective big blinds, straddle impact, and stack depth effects
3. **Estimate Parameters**: Calculates win rates (μ) and variance (σ²) with confidence intervals by stake level
4. **Simulate & Recommend**: Runs Monte Carlo simulations to assess risk of ruin and generate stake recommendations

### 📊 Key Outputs

- **Win Rate Estimates**: Per-hand expected value by stake with confidence intervals
- **Risk Analysis**: Risk of ruin probabilities over various time horizons
- **Stake Recommendations**: Clear guidance on which stakes to play based on current bankroll
- **Decision Memo**: A comprehensive summary suitable for future reference

### 🔄 Next Steps

1. **Update Data**: Replace sample data with your actual poker session logs
2. **Customize Parameters**: Adjust risk tolerance, bankroll size, and time horizons in `SIMULATION_CONFIG`
3. **Refine Estimates**: Add more sessions to improve statistical confidence
4. **Monitor Performance**: Re-run analysis periodically as you gather more data
5. **Implement Decisions**: Use recommendations to guide your stake selection

### 📝 Usage Notes

- **Data Quality**: The accuracy of recommendations depends on the quality and quantity of session data
- **Risk Management**: Always maintain proper bankroll management regardless of simulation results
- **Variance**: Live poker has high variance - be prepared for significant swings even at "safe" stakes
- **Continuous Improvement**: Regularly update your analysis as your game and bankroll evolve

### 🛠️ Customization

To adapt this notebook for your specific situation:

1. Modify the `create_sample_session_data()` function to load your actual data
2. Adjust the `SIMULATION_CONFIG` parameters for your risk tolerance and bankroll
3. Update the hands-per-hour estimates based on your typical game conditions
4. Customize the recommendation logic based on your personal criteria

This analysis provides a data-driven foundation for bankroll decisions, but should be combined with good judgment and proper risk management practices.